In [100]:
from neo4j import GraphDatabase
from CurriculumDB.Modelsn4j import *
import docx
import os

In [81]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = ("curriculum", "curriculumdb")

driver = GraphDatabase.driver(URI, auth=AUTH)
print(driver)
print(driver.verify_connectivity())
#help(driver.verify_connectivity)
factory = CurriculumFactory(driver, 'curriculumdb')

None


In [2]:
routes = {'BIMS': ["BMS","Biomedical Sciences"],
 'NEUR': ["BMS","Neuroscience"],
 'PHAR': ["BMS","Pharmacology"],
 'PHSC': ["BMS","Physiological Sciences"],
 'BIOLOGSCI': ["BIO","Biological Sciences"],
 'BIOC': ["BIO","Biochemistry"],
 'BSBI' : ["BIO","Biological Sciences (Bioinformatics)"],
 'BSPS': ["BIO","Biological Sciences (Plant Sciences)"],
 'MBIO': ["BIO","Microbiology"],
 'MOLG': ["BIO","Molecular Genetics"],
 'MOLB':["BIO","Molecular Biology"],
 'BCDD' : ["BIO","Biological Chemistry and Drug Discovery"]}

In [6]:
Programme.requiredParams


{'code': 'Programme code', 'name': 'Programme name'}

In [7]:
routeobj ={}
for p in routes:
    prog = factory.get_or_create_Element('Programme', code=p, name=routes[p][1])
    routeobj[p]=prog
    

In [13]:
doc = docx.Document("c:/Users/marti/Documents/LifeSciteaching/Curriculum/Programmes/UG/BSc Hons Biomedical Sciences Programme Specification 2324.docx")



In [110]:

class Node():
    '''Represents a Node in a Neo4j database. 
    Contains methods for accessing links, ensuring that on creation the 
    correct parameters are given etc.'''

    ntype='Node'
    
    requiredParams = {
        }
    optionalParams= {}
    
    def __init__(self, factory=None,  **kwparams):
        '''Initialiser. Checks required parameters are present'''
        if factory is None:
            raise Exception('Curriculum factory must be given as factory')
        self.factory = factory
        self.ntype=self.__class__.__name__.split('.')[-1]
        self.paramChecks={'default': self._checkDefault}
        self.params={}
        self.edges=[]
        self._get_labels()
        for par in self.requiredParams:
            if not kwparams.get(par):
                print(par, kwparams)
                raise InsufficientParameterSpecException(f'Missing parameter <{par}> of type <{self.requiredParams[par]}>')
            if not self.paramChecks.get(par,self._checkDefault)(kwparams[par]):
                raise BadParameterException(f'Parameter <{par}> should be of type <{self.requiredParams[par]}>')
            self.params[par]=kwparams[par]
        for par in self.optionalParams:
            if par not in kwparams:
                continue
            if not self.paramChecks.get(par,self._checkDefault)(kwparams[par]):
                raise BadParameterException(f'Parameter <{par}> should be of type <{self.requiredParams[par]}>')
            self.params[par]=kwparams[par]
        if 'elementID' not in kwparams:
            self._create_node()
        else:
            self.element_id=kwparams['elementID']
            self.edges =self.getEdges()
    def __str__(self):
        '''Returns a string represntation of the node'''
        params = ", ".join([f'{k}: {self.params[k]}' for k in self.requiredParams])
        return f'{self.ntype}: [{params}] ID: {self.element_id}'
    
    def __repr__(self):
        return self.__str__()
                            
    def _create_node(self):
        '''Internal method that creates a new instance of the node, if it does not exist'''
        paramlist = ", ".join([f'{k}: ${k}' for k in self.params])
        paramlist = "{"+paramlist+"}"
        nodelabels = ':'.join(self.labels)
        query= f"MERGE (a:{nodelabels} {paramlist}) RETURN a"
        records,summary, keys = self.factory.db.execute_query(query, self.params, database_=self.factory.dbname)   
        self.element_id = records[0].items()[0][1].element_id
            
    def _checkDefault(self, value):
        '''default placeholder for parameter value checks. Returns True'''
        return True
    
    def _checkAY(self, value):
        '''Checks for a correctly formatted Academic year as:
            2425
            24/25
            2024/5'''
        #TODO 
        return True
    def getparam(self, prop, default=None):
        '''
        Returns a parameter value (default if not found)

        Parameters
        ----------
        prop : property name
            
        default : TYPE, optional
             The default is None.

        Returns
        -------
        Property value
            
        '''
        return self.params.get(prop,default)
    
    def setparam(self, key, value):
        '''
        Sets the parameter key to value. No type checking.

        Parameters
        ----------
        key : legal value for a dictionary key
            
        value : Value to store.
        
        Returns
        -------
        None.

        '''
        self.params[key]=value
        self.update()
    
    def getEdges(self,relation=None,**kwargs):
        '''retrieves edges connected to or from the Node.
        Relation is the type of relation to retrieve (default is all)
        other arguments are applied as matching parameters'''
        edges=[]
        filterstring=''
        filterparams={}
        for kw in kwargs:
            if kw != 'id':
                filterstring += f"'{kw}': ${kw},"
                filterparams[kw] = kwargs[kw]
        rel=''
        if relation:
            rel=f":{relation}"
        filterparams['id'] = self.element_id
        if filterstring:
            filterstring = "{"+filterstring+"}"
        records, summary, keys = self.factory.db.execute_query(
        f"MATCH (p: {self.ntype} {{id: $id }}) -[e{rel} {filterstring}]-(q) RETURN e,q",
        filterparams,
        routing_=neo4j.RoutingControl.READ,  # or just "r"
        database_=self.factory.dbname)
        for edge in records:
            relation = edge.items[0][1]
            target = edge.items[1][1]
            edges.append({'source': self,'edge': relation, 'target': target})
        return edges
    def isCurrent(self, year):
        '''
        If the edge has a start/end year, check if the given year is the period in which the relation is valid

        Returns
        -------
        Boolean: True if the eyar is between start and end (inclusive), after start if no end, or True if no start

        '''
        
        if self.params.get('startyear'):
            if AcademicYear(year)< AcademicYear(self.params.get('startyear')):
                return False
            if self.params.get('endyear'):
                if AcademicYear(year)> AcademicYear(self.params.get('endyear')):
                    return False
        return True
    
    
    def toDict(self):
        retp ={'element_id': self.element_id}
        for p in self.params:
            retp[p]=self.params[p]
        return retp
    
    def update(self, **kwargs):
        '''
        Update named parameters for a Node

        Parameters
        ----------
        **kwargs : key,value parameter set
            any key,value parameters. Existing values will be overwritten.
            No type checking

        Returns
        -------
        None.

        '''
        paramlist =[]
        if not kwargs:
            return
        for k in kwargs:
            if k !='id':
                paramlist.append( f'a.{k}= ${k}')
                self.params[k]=kwargs[k]
        kwargs['id'] = self.element_id    
        paramlist = "{"+paramlist+"}"
        query= f"MERGE (a:{self.ntype}) WHERE elementID(a) = $id SET {', '.join(paramlist)}"
        records,summary, keys = self.factory.db.execute_query(query,kwargs, database_=self.factory.dbname)   
        
    def _get_labels(self):
        self.labels = [x.__name__ for x in globals()[self.__class__.__name__].__mro__]
        self.labels.remove('object')
        self.labels.remove('Node')
        
    
    
class Edge():
    '''Represents an edge in a Neo4J database'''
    requiredParameters={
        'relation': 'relationship type',
        'source': 'Node ',
        'target': 'Node '}
    optionalParams={
        'startyear': 'Academic year start',
        'endyear': 'Academic year to'
        }
    def __init__(self, factory, **kwargs):
        '''Edge between two nodes. Must specify source, target and relationship type.'''
        for kw in self.requiredParameters:
            if kw not in kwargs:
                raise InsufficientParameterSpecException(f"{kw} not specified in Edge constructor")
            self.params[kw]=kwargs[kw]
    def set_expiry(self, expirydate):
        '''
        Sets the 'to' parameter with an academic year for when this relation was withdrawn. 
        The date specified will be the last academic year for which it was valid.

        Parameters
        ----------
        expirydate : Academic year
            Academic year is an academic year object or text description of it.

        Returns
        -------
        None.

        '''
        self.params['to'] =AcademicYear(expirydate)
    
    def isCurrent(self, year):
        '''
        If the edge has a start/end year, check if the given year is the period in which the relation is valid

        Returns
        -------
        Boolean: True if the eyar is between start and end (inclusive), after start if no end, or True if no start

        '''
        
        if self.params.get('startyear'):
            if AcademicYear(year)< AcademicYear(self.params.get('startyear')):
                return False
            if self.params.get('endyear'):
                if AcademicYear(year)> AcademicYear(self.params.get('endyear')):
                    return False
        return True
    def getSource(self):
        '''
        Returns a Python object representing the source node

        Returns
        -------
        None.

        '''
        
class CurriculumFactory():
    
    def __init__(self, db, dbname='curriculum'):
        self.db = db
        self.dbname = dbname
        self.programmecache = {}
        self.modulecache = {}
        self.activitycache = {}
        self.personcache = {}
        
    def get_connection(self):
        return self.db
    
    @lru_cache    
    def get_element_by_ID(self, elementID):
        '''
        Retrieves an element by ID and creates the appropriate node for it, if it is a node.
        
        Parameters
        ----------
        elementID : Internal neo4j element ID
            text string uniquely identifying the object in the database.

        Returns
        -------
        Object of correct class, if available or None if not found.

        '''
        erecords, _, _ = self.db.execute_query(
            "MATCH (a ) where elementId(a)=$id"
            "RETURN a", id=elementID,
             database_=self.dbname,
        )
        if not erecords :
            return
        nodeclass = list(erecords[0].items()[0][1].labels())[0]
        params = dict(erecords[0].items()[0][1])
        params['elementID']=elementID
        if nodeclass in globals():
            return globals()[nodeclass](**params)
        
      
    def get_or_create_Element(self, ElementName,**kwargs):
        '''
        Generic get and create method. This requires all elements to have a unique id under the property code

        Parameters
        ----------
        ElementName : Name of element to create, eg Programme
            This will directly map to the class name.
        **kwargs : Parameters to use. These will be checked against every class.
            

        Returns
        -------
        Element of type ElementName on success

        '''
        
        elem= None
        
        if ElementName not in globals():
            return
        ElementClass=globals()[ElementName]
        if kwargs.get('elementID',None):
            elem = self.get_element_by_id(kwargs['elementID'])
            return elem
        if kwargs.get('code'):
            try:
                result,_,_=self.db.execute_query(f'MATCH (p:{ElementName} {{code: $name}} ) return p', name=kwargs['code'], database_=self.dbname)
                if result:
                    params=dict(result[0].items()[0][1])
                    params['elementID']=result[0]['p'].element_id
                    elem = ElementClass(self, **params)
                    return elem
            except Exception as e:
                e.add_note(str(params))
                raise e
                
        for p in ElementClass.requiredParams:
            if p not in kwargs:
                raise InsufficientParameterSpecException(f'Not enough parameters specified for {ElementName}: missing{p}')
        
        elem = ElementClass(self, **kwargs)
        return elem


    def get_all_elements(self, ElementName, **params):
        '''
        Returns all elements of type ElementName

        Parameters
        ----------
        ElementName : name of an element
            This can be any element type for which a model exists.
        **params : Keyword parameters to use in the search
            Exact matches only at present.

        Returns
        -------
        List of elements

        '''
        
        elements = []
        if ElementName not in globals():
            return elements
        paramdetails =[]
        paramtext=''
        for p in params:
            paramdetails.append(f'{p}: ${p}')
        if paramdetails:
            paramtext= f'{ {",".join(paramdetails)} }'
        erecords, _, _ = self.db.execute_query(
            f"MATCH (a :{ElementName} {paramtext}) RETURN a",
            params,
             database_=self.dbname,
        )
        try:
            if not erecords :
                return elements
            for item in erecords:
                elem=item.items()[0][1]
                nodeclass = list(elem.labels)[0]
                params = dict(elem)
                params['elementID']=elem.element_id
                if nodeclass in globals():
                    elements.append( globals()[nodeclass](self, **params))
            return elements  
        except Exception as e:
            print(e)
            return erecords
    def getElementsForElement(self, element, target, max_steps=2,relation=None):
        '''
        Retreive all elements of type Target linked to element,optionally by relation (or all relations) 
        at a maximum distance of max_steps (default 2)
        
        Parameters
        ----------
        element : TYPE
            DESCRIPTION.
        target : TYPE
            DESCRIPTION.
        max_steps: integer
            Maximum link number to explore (default 2)
        relation: text
            Limit relations to those of type relation
        Returns
        -------
        List of Node type objects.
        '''
        if target not in globals():
            return []
        cypher = f"MATCH (e) --{{1,{max_steps}}} (thing:$target) where elementID(e)=$id RETURN DISTINCT thing"
        result,_,_=self.db.execute_query(cypher, target=target, id=element.element_id,database_=self.dbname)
        elems = []
        if result:
            for elem in result:
                item = elem.items()[0][1]
                params =dict(item)
                params['elementID'] = item.element_id
                elems.append(globals()[target](self, **params))
        return elems
         
class Programme(Node):
    
    DRAFT = 0
    CURRENT = 1
    ARCHIVED = 2
    WITHDRAWN =3
    ntype='Programme'
    requiredParams = {
        'code': 'Programme code',
        'name': 'Programme name'
        }
    optionalParams= {
        'startyear':'First academic year',
        'endyear': 'Last academic year',
        'school':'School which manages the Programme'}
    def __init__(self, factory, **params ):
        '''
        Create a Programme instance.

        Parameters
        ----------
        factory - database connection and cache for Programme
        **params : 
            Create or replace table Programme (
        '''
        super().__init__(factory,**params)
        
        self.modules = {}
        self.ILO = []
        if self.element_id:
            self.loadmodules()
            self.loadILO()
        

    
    
    
    def loadmodules(self):
        '''
        Loads module links from the database to the modules list. Does not create now entries in the database.

        Returns
        -------
        None.

        '''
        self.modules={}
        cypher = "MATCH (p:Programme ) -[a]-(b:Module) WHERE elementID(p) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation = t.items()[0][1]
            target = dict(t.items()[1][1])
            target['element_id'] = t.items()[1][1].element_id
            if target['code'] not in self.modules:
                self.modules[target['code']]=[]
            self.modules[target['code']].append({'relation': {'type': relation.type, 'params':dict(relation)}, 
                                                        'target':{ 'moduleID': target['element_id'],'params':target}})
            
        

        
    def loadILO(self):
        '''
        Loads ILO links from the database. Does not create new links or ILOs.

        Returns
        -------
        None.

        '''
        self.ILO = {}
        cypher = "MATCH (p:Programme ) -[a]-(b:ProgrammeILO) WHERE elementID(p) = $id RETURN a,b"
        records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id, database_=self.factory.dbname)
        for t in records:
            relation, target = t.items()[0:2]
            
            self.ILO[target.element_id]= (dict(target), dict(relation))
        
 
        
    def withdraw(self, year):
        '''
        Set the Programme to WITHDRAWN. Only possible if there are no future versions.

        Returns
        -------
        None.

        '''
        if AcademicYear(year) and AcademicYear(year) > AcademicYear(self.params['startyear']):
            self.params['endyear']=year
            cypher = 'MATCH (p:Programme) where elementID(p)=$id SET p.endyear=$year'
            records,_,_ =self.factory.db.execute_query(cypher, id=self.element_id,year=year, database_=self.factory.dbname)
        

    def map_module(self, module, optional=0, year=None, remove=False):
        '''
        Add a module to the Programme, updating if exisitng  already appended.

        Parameters
        ----------
        module : Module object
            DESCRIPTION.
        optional: Boolean
        remove: Academic Year for last instance
        Returns
        -------
        None.

        '''
        relations=('IS_CORE','IS_ELECTIVE')
        relation= relations[int(bool(optional))]
        cypher1 = 'MATCH (p:Programme) where elementID(p)=$pid \n MATCH (m:Module) where elementID(m)=$mid \n merge (p) <-[b:{relation}]-(m)  return p,b,m'
        
        if module.params['code'] in self.modules:
            if remove:
                
                records,_,_ =self.factory.db.execute_query(cypher1.format(relation=relation), 
                                                           pid=self.element_id,year=year, 
                                                           mid=module.element_id,
                                                           database_=self.factory.dbname)
                for r in self.modules[module.params['code']]:
                    if r['relation']['type']==relation:
                        r['relation']['params']['endyear']=year
            else:
                for r in self.modules[module.params['code']]:
                    if r['relation']['type']!=relation:
                        r['relation']['params']['endyear']=year
                        records,_,_ =self.factory.db.execute_query(cypher1.format(relation=relations[(int(bool(optional))+1)%2]), 
                                                                   pid=self.element_id,year=year, 
                                                                   mid=module.element_id,
                                                                   database_=self.factory.dbname)
         
        if not remove:
            if module.params['code'] not in self.modules:
                 self.modules[module.params['code']]=[]
            cypher2 =f'MATCH (p:Programme) where elementID(p)=$pid MATCH (m:Module)  where elementID(m)=$mid MERGE (p)<-[:{relation} {{startyear:$year }}] - (m) return m,p'
            records,_,_ =self.factory.db.execute_query(cypher2, 
                                                        pid=self.element_id,year=year, 
                                                        mid=module.element_id,
                                                        database_=self.factory.dbname)
        self.loadmodules()
           
                                           
               
           
           
    def map_ilo(self, ilo, year, remove=False):
        '''
        Associates or updates a Programme ILO mapping to a Programme.

        Parameters
        ----------
        ilo : ProgrammeILO 
            Programme ILO object
        year : text describing the academic year
            Must be either the start year, or for removing an ILO the last year for which it will be relevant.
        remove : Boolean, optional
            Flag to say whether the ILO should be dissociated from teh Programme. The default is False.

        Returns
        -------
        None.

        '''
        
        if not AcademicYear(year):
            raise UnparseableYearException(f'Cannot parse year value {year}')
        if ilo.element_id in self.ILO:
            if remove:
                self.ILO[ilo.element_id]['endyear']=year
                cypher="MATCH (p: Programme) -[b]- (i:ProgrammeILO) WHERE elementID(p) =$pid AND elementID(i) =$iid SET b.endyear=$year"
                records,_,_ = self.factory.db.execute_query(cypher,pid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
            return
        if remove:
            return
        cypher = "MERGE (p:Programme) -[b:HAS_ILO {startyear:$year}]->(i:ProgrammeILO) where elementID(p)=$pid AND elementID(i) = $iid RETURN b"
        records,_,_ = self.factory.db.execute_query(cypher,pid=self.element_id, iid=ilo.element_id, year=year, database_=self.factory.dbname)
        if records:
            relation=records[0].items()[0][1]
            self.ILO[ilo.element_id]=(dict(ilo),dict(relation))



In [111]:
factory = CurriculumFactory(driver, 'curriculumdb')

In [58]:
modmap = []
structure ={}
for t in doc.tables:
    for r in range(len(t.rows)):
        #print(len(t.rows), len(t.row_cells(r)), t.row_cells(r)[0].text,[c.tables for c in t.row_cells(r)])
        structure[t.row_cells(r)[0].text.split()[0]] = t.row_cells(r)
tc=0
for m in structure['2.10'][0].tables:
    coretype=['Core','Elective'][tc%2]
    tc+=1
    for r in range(1,len(m.rows)):
        print([coretype]+[c.text for c in m.row_cells(r)])
        modmap.append([coretype]+[c.text for c in m.row_cells(r)])

['Core', 'BS11009', 'The Building Blocks of Life', '1', '7', '20', 'SLS']
['Core', 'BS11008', 'Core Skills in the Life Sciences 1A', '1', '7', '20', 'SLS']
['Core', 'BS12011', 'Building the Organism', '1', '7', '20', 'SLS']
['Core', 'BS12010', 'Core Skills in the Life Sciences 1B', '1', '7', '20', 'SLS']
['Core', 'BS21001', 'Statistics and Experimental Design', '2', '8', '10', 'SLS']
['Core', 'BS21002', 'Cellular Communication', '2', '8', '10', 'SLS']
['Core', 'BS21012', 'Core Skills in the Life Sciences 2A', '2', '8', '20', 'SLS']
['Core', 'BS22001', 'Human Physiology and Pharmacology', '2', '8', '20', 'SLS']
['Core', 'BS22002', 'Biomolecular Mechanisms', '2', '8', '20', 'SLS']
['Core', 'BS22003', ' Core Skills in the Life Sciences 2B', '2', '8', '20', 'SLS']
['Core', 'BS31013', 'Biomembranes', '3', '9', '15', 'SLS/SoM']
['Core', 'BS31016', 'Practical Techniques in Biomedical Sciences', '3', '9', '15', 'SLS/SoM']
['Core', 'BS31019', 'Regulatory Physiology & Pharmacology', '3', '9', '1

In [112]:
mods = factory.get_all_elements('Module')
modules ={m.params['code']:m for m in mods} 

In [48]:
modules


{'BS11005': Module: [code: BS11005, name: Numeracy, Chemistry and Physics for the Biological and Biomedical Sciences, credits: 20, scqflevel: 7, shelevel: 1, semester: 1] ID: 4:04f9907a-1dd4-4d8a-a4fe-7ccb993179fa:12,
 'BS11006': Module: [code: BS11006, name: The Poison Pen, credits: 20, scqflevel: 7, shelevel: 1, semester: 1] ID: 4:04f9907a-1dd4-4d8a-a4fe-7ccb993179fa:13,
 'BS11008': Module: [code: BS11008, name: Core Skills in the Life Sciences 1A, credits: 20, scqflevel: 7, shelevel: 1, semester: 1] ID: 4:04f9907a-1dd4-4d8a-a4fe-7ccb993179fa:14,
 'BS11009': Module: [code: BS11009, name: The Building Blocks of Life, credits: 20, scqflevel: 7, shelevel: 1, semester: 1] ID: 4:04f9907a-1dd4-4d8a-a4fe-7ccb993179fa:15,
 'BS12005': Module: [code: BS12005, name: Science and Society, credits: 20, scqflevel: 7, shelevel: 1, semester: 2] ID: 4:04f9907a-1dd4-4d8a-a4fe-7ccb993179fa:16,
 'BS12008': Module: [code: BS12008, name: Introduction to Scientific Enterprise, credits: 20, scqflevel: 7, she

In [55]:
{s:structure[s][-1].text for s in structure}

{'Applicability': 'From 2023-24',
 'Section': 'Section 7: University management information (to be completed by the University’s Quality and Academic Standards office)',
 'Heading': 'Details',
 '1.1': 'Bachelor of Science (Hons) in Biomedical Sciences',
 '1.2': 'Biomedical Sciences\n(B1) Anatomy, physiology & pathology\n(B2) Pharmacology, toxicology & pharmacy\n(C0) Broadly-based programmes within biological sciences\n(C1) Biology\n(C9) Others in Biological Sciences\n',
 '1.3': 'B900 Biomedical Sciences\nB901 Biomedical Sciences with year in industry\n',
 '1.4': 'University of Dundee',
 '1.5': 'University of Dundee',
 '1.6': 'English',
 '1.7': 'SCQF Level 10',
 '1.8': '',
 '1.9': '\n\n',
 '1.10': '\n',
 '1.11': 'Details of the programme aims, indicative content and intended learning outcomes are provided in the Programme Specification.',
 '1.12': 'Details about the modules undertaken by the students are described in this Programme Specification, section 2.10',
 '1.13': 'The University’

In [95]:
programmes={'BIMS':factory.get_or_create_Element('Programme', code='BIMS',name=structure['1.1'][-1].text)}

In [57]:
help(programmes['BIMS'])

Help on Programme in module __main__ object:

class Programme(Node)
 |  Programme(factory, **params)
 |  
 |  Represents a Node in a Neo4j database. 
 |  Contains methods for accessing links, ensuring that on creation the 
 |  correct parameters are given etc.
 |  
 |  Method resolution order:
 |      Programme
 |      Node
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, factory, **params)
 |      Create a Programme instance.
 |      
 |      Parameters
 |      ----------
 |      factory - database connection and cache for Programme
 |      **params : 
 |          Create or replace table Programme (
 |  
 |  loadILO(self)
 |      Loads ILO links from the database. Does not create new links or ILOs.
 |      
 |      Returns
 |      -------
 |      None.
 |  
 |  loadmodules(self)
 |      Loads module links from the database to the modules list. Does not create now entries in the database.
 |      
 |      Returns
 |      -------
 |      None.
 |  
 |  map_

In [98]:
for m in modmap:
    if modules.get(m[1].strip('*')):
        programmes['BIMS'].map_module(modules[m[1].strip('*')],m[0]=='Elective', year='23/24')

In [103]:
docs = {
    'BIO':'BSc Hons Biologsci Programme Specification QASv3.docx',
'BIMS': 'BSc Hons Biomedical Sciences Programme Specification 2324.docx',
 'NEUR': 'BSc Hons Neuroscience Programme Specification QASv8.docx',
 'PHAR': 'BSc Hons Pharmacology Programme Specification QASv6.docx',
 'PHYS': 'BSc Hons Physiological Sciences Programme Specification QASv6.docx'
    
}
    

progdir='c:/Users/marti/Documents/LifeSciteaching/Curriculum/Programmes/UG/'

In [113]:
for route in ['BIMS','NEUR','PHAR','PHYS']:
    doc = docx.Document(os.path.join(progdir,docs[route]))
    modmap = []
    structure ={}
    for t in doc.tables:
        for r in range(len(t.rows)):
            #print(len(t.rows), len(t.row_cells(r)), t.row_cells(r)[0].text,[c.tables for c in t.row_cells(r)])
            structure[t.row_cells(r)[0].text.split()[0]] = t.row_cells(r)
    tc=0
    for m in structure['2.10'][0].tables:
        coretype=['Core','Elective'][tc%2]
        tc+=1
        for r in range(1,len(m.rows)):
            print([coretype]+[c.text for c in m.row_cells(r)])
            modmap.append([coretype]+[c.text for c in m.row_cells(r)])
    programmes[route]=factory.get_or_create_Element('Programme', code=route,name=structure['1.1'][-1].text)
    for m in modmap:
        if modules.get(m[1].strip('*')):
            programmes[route].map_module(modules[m[1].strip('*')],m[0]=='Elective', year='23/24')

['Core', 'BS11009', 'The Building Blocks of Life', '1', '7', '20', 'SLS']
['Core', 'BS11008', 'Core Skills in the Life Sciences 1A', '1', '7', '20', 'SLS']
['Core', 'BS12011', 'Building the Organism', '1', '7', '20', 'SLS']
['Core', 'BS12010', 'Core Skills in the Life Sciences 1B', '1', '7', '20', 'SLS']
['Core', 'BS21001', 'Statistics and Experimental Design', '2', '8', '10', 'SLS']
['Core', 'BS21002', 'Cellular Communication', '2', '8', '10', 'SLS']
['Core', 'BS21012', 'Core Skills in the Life Sciences 2A', '2', '8', '20', 'SLS']
['Core', 'BS22001', 'Human Physiology and Pharmacology', '2', '8', '20', 'SLS']
['Core', 'BS22002', 'Biomolecular Mechanisms', '2', '8', '20', 'SLS']
['Core', 'BS22003', ' Core Skills in the Life Sciences 2B', '2', '8', '20', 'SLS']
['Core', 'BS31013', 'Biomembranes', '3', '9', '15', 'SLS/SoM']
['Core', 'BS31016', 'Practical Techniques in Biomedical Sciences', '3', '9', '15', 'SLS/SoM']
['Core', 'BS31019', 'Regulatory Physiology & Pharmacology', '3', '9', '1

['Elective', 'BS42013', 'Advanced Cell Signalling', '4', '10', '15', 'SLS']
['Elective', 'BS42014', 'Nutrients & Metabolic Disease', '4', '10', '15', 'SLS/SoM']
['Elective', 'BS42019', 'Cardiovascular Pharmacology', '4', '10', '15', 'SLS/SoM']
['Elective', 'BS42021', 'Heart & Circulation', '4', '10', '15', 'SLS/SoM']
['Elective', 'BS42027', 'Cancer Pharmacology & Treatment', '4', '10', '15', 'SLS/SoM']
['Elective', 'BS42028', 'Pharmacology and Treatment of Metabolic Disease ', '4', '10', '15', 'SLS/SoM']
['Core', 'BS11009', 'The Building Blocks of Life', '1', '7', '20', 'SLS']
['Core', 'BS11008', 'Core Skills in the Life Sciences 1A', '1', '7', '20', 'SLS']
['Core', 'BS12011', 'Building the Organism', '1', '7', '20', 'SLS']
['Core', 'BS12010', 'Core Skills in the Life Sciences 1B', '1', '7', '20', 'SLS']
['Core', 'BS21001', 'Statistics and Experimental Design', '2', '8', '10', 'SLS']
['Core', 'BS21002', 'Cellular Communication', '2', '8', '10', 'SLS']
['Core', 'BS21012', 'Core Skills i